In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from pymongo import MongoClient

In [2]:
df = pd.read_excel("/home/waris/Github/tupras-analysis/alarm.xlsx")


In [3]:
client = MongoClient()
db = client["Tupras"]
collectionRawData = ["Raw-Data"]
collectionT = db["Alarm-Preprocessing-Step1"]

In [6]:
# sources = df["SourceName"].values
# start_time = df["ActiveTime"].values
# end_time = df["EventTime"].values
# message = df["Message"].values
# print(sources, start_time, end_time, message)
# print(type(end_time[0]), type(message[0]))
# print(end_time[0]-start_time[0])

df.


SyntaxError: invalid syntax (<ipython-input-6-4cd7499e222e>, line 9)

Preprocessing Step 1

In [ ]:
def getTargetEventList(start,stime,etime):
    tList = []

    for i in range(start,len(start_time)):
                
        if (stime < start_time[i]):
            if (start_time[i] < etime):
                if (end_time[i] < etime):
                    tList.append({"target": sources[i], "target-start-time": np.datetime_as_string(start_time[i]), "target-end-time": np.datetime_as_string(end_time[i])}) 
            else:
                break
        else:
            break

    return tList

In [ ]:
# for i, s in enumerate(sources):
#     tList = getTargetEventList(i+1,start_time[i], end_time[i])
#     if len(tList) > 0:
#         # print("Iteration", i, "Number of alarms", len(tList))
#         collectionT.insert_one({"source":s, "source-start-time": np.datetime_as_string(start_time[i]), "source-end-time":np.datetime_as_string(end_time[i]),"tList": tList})


In [ ]:
def intersection1(lst1, lst2): 
    lst3 = [d["target"] for d in lst1 if d["target"] in lst2] 
    return list(set(lst3)) 

def intersection2(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

def convertToProperList(l):
    temp = []

    for d in l:
        temp.append(d["target"])
    
    return list(set(temp))


In [ ]:
def test(s):
    cursor_target_lists = collectionT.find({"source": s}, { "tList.target": 1,"_id": 0 })
    intersection_list = []
    flag = True
    duplicates = 0
    for d in cursor_target_lists:
        list1 = d["tList"]
        
        if len(list1)>8:
            if flag:
                flag = False       
                intersection_list = convertToProperList(list1)
            
            temp_len_intersection = len(intersection_list) 
            temp_len_list1 = len(list1) 
            list1 = convertToProperList(list1)
            intersection_list = intersection2(list1, intersection_list)
            duplicates += (temp_len_list1 - len(list1))
            print("Orignal Length :{l}, after len: {al}, duplicates: {dup}".format(l=temp_len_list1, al=len(list1), dup=duplicates))
            if (len(intersection_list)>0):
                print("List1: {l1}, List2: {l2}, Intersection: {l3}".format(l1= len(list1), l2 = temp_len_intersection, l3= len(intersection_list)))
        
            
    return intersection_list, duplicates



In [ ]:
## Tacking Intersection
sensors = collectionT.distinct("source")


In [ ]:

DUPLICATES = 0

for i, s in enumerate(sensors):
    print("==================== {}============================".format(i))
    _ , d = test(s)
    print("Duplicates",  d)
    DUPLICATES += d 

print("Total Duplicates", DUPLICATES)

In [ ]:
col